# Volatility Indices Calculation

This notebook explains how the module *xvbt_calc* calculates the XVBT, AXVBT and GXVBT indices using data from Deribit.

In [2]:
import calendar
import numpy as np
import openapi_client as dbitApi
import pandas as pd

from datetime import datetime

### Utility functions for time calculations, Deribit API and dataframe formatting

In [201]:
def format_datetime_to_expiry(date):
    return datetime.strftime(date, '%-d%b%y').upper()

def get_near_next_terms(now):
    c = calendar.Calendar(firstweekday=calendar.MONDAY)
    
    this_month_cal = c.monthdatescalendar(now.year, now.month)
    this_fridays = [datetime(day.year, day.month, day.day, 8, 0, 0) 
                    for week in this_month_cal for day in week 
                    if day.weekday() == calendar.FRIDAY and day.month == now.month 
                    and datetime(day.year, day.month, day.day, 8, 0, 0) >= now]
    
    next_year = now.year if now.month < 12 else now.year + 1
    next_month = now.month + 1 if now.month < 12 else 1
    
    next_month_cal = c.monthdatescalendar(next_year, next_month)
    next_fridays = [datetime(day.year, day.month, day.day, 8, 0, 0) 
                    for week in next_month_cal for day in week 
                    if day.weekday() == calendar.FRIDAY and day.month == next_month 
                    and datetime(day.year, day.month, day.day, 8, 0, 0) >= now]
    
    fridays = this_fridays + next_fridays
    
    near_term, next_term = fridays[0], fridays[1]
        
    return (format_datetime_to_expiry(near_term), format_datetime_to_expiry(next_term), near_term, next_term)

def get_index(currency='BTC'):
    try:
        index_result = api.public_get_index_get(currency)['result'][currency]
        return index_result
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_instruments_with_expiry(expiry, currency='BTC', kind='option', expired='false'):
    try:
        instrument_result = api.public_get_instruments_get(currency, kind=kind, expired=expired)['result']
        return [instrument['instrument_name'] for instrument in instrument_result if expiry in instrument['instrument_name']]
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_ticker(instrument):
    try:
        instrument_result = api.public_ticker_get(instrument)['result']
        return instrument_result
    except dbitApi.exceptions.ApiException as e:
        print(e)
        #logger.exception('Exception when calling MarketDataApi->public_get_instruments_get!')
        exit()

def get_bids_asks(near_list, next_list):
    near_calls = dict()
    near_puts = dict()
    next_calls = dict()
    next_puts = dict()

    for instrument in near_list:
        data = get_ticker(instrument)
        best_bid, best_ask = data['best_bid_price'], data['best_ask_price']
        strike, cp = int(instrument.split('-')[2]), instrument.split('-')[3]

        if cp == 'C':
            near_calls[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        elif cp == 'P':
            near_puts[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        else:
            print(f'Error {instrument}')

    for instrument in next_list:
        data = get_ticker(instrument)
        best_bid, best_ask = data['best_bid_price'], data['best_ask_price']
        strike, cp = int(instrument.split('-')[2]), instrument.split('-')[3]

        if cp == 'C':
            next_calls[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        elif cp == 'P':
            next_puts[strike] = {'best_bid': best_bid, 'best_ask': best_ask}
        else:
            print(f'Error {instrument}')

    near_calls_df = pd.DataFrame.from_dict(near_calls, orient='index').sort_index().replace(0, np.nan)
    near_puts_df = pd.DataFrame.from_dict(near_puts, orient='index').sort_index().replace(0, np.nan)
    next_calls_df = pd.DataFrame.from_dict(next_calls, orient='index').sort_index().replace(0, np.nan)
    next_puts_df = pd.DataFrame.from_dict(next_puts, orient='index').sort_index().replace(0, np.nan)

    return near_calls_df, near_puts_df, next_calls_df, next_puts_df

## XVBT Implementation

Replication of CBOE VIX calculation.

Near and next term expiries are defined as the next two Fridays respectively. Bid/ask data for all strike puts and calls are retrieved from Deribit for these expiries.

In [190]:
api = dbitApi.MarketDataApi()

now = datetime.now()
near_expiry, next_expiry, near_datetime, next_datetime = get_near_next_terms(now)

print(near_expiry, next_expiry)

24APR20 1MAY20


In [202]:
near_instruments = get_instruments_with_expiry(near_expiry)
next_instruments = get_instruments_with_expiry(next_expiry)

near_calls_df, near_puts_df, next_calls_df, next_puts_df = get_bids_asks(near_instruments, next_instruments)

In [203]:
near_calls_df

,best_bid,best_ask
3000,0.5655,0.5885
3500,0.4930,0.5200
4000,0.4210,0.4520
4500,0.3485,0.3840
5000,0.2760,0.3165
5500,0.2040,0.2490
6000,0.1405,0.1830
6250,0.1130,0.1365
6500,0.0885,0.0925
6750,0.0575,0.0625


### Step 1: Select the options to be used in the VIX Index calculation

Call and put prices are computed as the average of the respective bid and ask prices. The strike at which the call and put price difference is found to calculate forward prices and separation strikes.

In [204]:
near_prices = pd.DataFrame(index=near_calls_df.index)
near_prices['call_price'] = (near_calls_df['best_bid'] + near_calls_df['best_ask']) / 2
near_prices['put_price'] = (near_puts_df['best_bid'] + near_puts_df['best_ask']) / 2
near_prices['abs_diff'] = abs(near_prices['call_price'] - near_prices['put_price'])

min_near_strike = near_prices['abs_diff'].idxmin()
min_near_diff = near_prices.loc[min_near_strike].abs_diff

next_prices = pd.DataFrame(index=next_calls_df.index)
next_prices['call_price'] = (next_calls_df['best_bid'] + next_calls_df['best_ask']) / 2
next_prices['put_price'] = (next_puts_df['best_bid'] + next_puts_df['best_ask']) / 2
next_prices['abs_diff'] = abs(next_prices['call_price'] - next_prices['put_price'])

min_next_strike = next_prices['abs_diff'].idxmin()
min_next_diff = next_prices.loc[min_next_strike].abs_diff

near_prices

,call_price,put_price,abs_diff
3000,0.57700,NaN,NaN
3500,0.50650,NaN,NaN
4000,0.43650,NaN,NaN
4500,0.36625,0.00075,0.36550
5000,0.29625,0.00100,0.29525
5500,0.22650,0.00175,0.22475
6000,0.16175,0.00275,0.15900
6250,0.12475,0.00425,0.12050
6500,0.09050,0.00675,0.08375
6750,0.06000,0.01150,0.04850


The XVBT index is set to have a constant maturity of seven days and a yield rate of zero (which should not make a difference to calculations - refer to Alexander paper page 9). This is used to calculate forward prices f1, f2 and separation strikes k0_1, k0_2.

In [222]:
const_mature_days = 7
R = 0

n1 = (near_datetime - now).total_seconds() / 60
n2 = (next_datetime - now).total_seconds() / 60
nY = 525600
n = const_mature_days * 24 * 60

t1 = n1/nY
t2 = n2/nY

# Compute forward prices and at-the-money strikes
f1 = min_near_strike + np.e**(R*t1) * min_near_diff
k0_1 = max([strike for strike in near_prices.index if strike <= min_near_strike])

f2 = min_next_strike + np.e**(R*t2) * min_next_diff
k0_2 = max([strike for strike in next_prices.index if strike <= min_next_strike])

print(k0_1, f1, k0_2, f2)

7000 7000.0135 7000 7000.01075


Out of the money calls and puts are found by using the calculated separation strikes and excluding at the money strikes.

In [223]:
near_otm_puts_df = near_puts_df.loc[:k0_1][:-1]
near_otm_calls_df = near_calls_df.loc[k0_1:][1:]
next_otm_puts_df = next_puts_df.loc[:k0_2][:-1]
next_otm_calls_df = next_calls_df.loc[k0_2:][1:]

In [224]:
near_otm_puts_df

,best_bid,best_ask
3000,NaN,0.0005
3500,NaN,0.0005
4000,NaN,0.0005
4500,0.0005,0.0010
5000,0.0005,0.0015
5500,0.0015,0.0020
6000,0.0025,0.0030
6250,0.0035,0.0050
6500,0.0065,0.0070
6750,0.0105,0.0125


In [225]:
near_otm_calls_df

,best_bid,best_ask
7250,0.0175,0.0195
7500,0.0090,0.0105
7750,0.0050,0.0060
8000,0.0035,0.0040
8250,0.0025,0.0035
8500,0.0015,0.0030
9000,0.0010,0.0020
10000,NaN,0.0010
11000,NaN,0.0005
12000,NaN,0.0010


Strikes following two consecutive bid prices and strikes with zero bids are excluded.

In [209]:
near_otm_puts_df = near_otm_puts_df.sort_index(ascending=False)
near_otm_puts_df = near_otm_puts_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
near_otm_puts_df['zero_bid_cumsum'] = near_otm_puts_df['zero_bid'].cumsum()
near_otm_puts_df = near_otm_puts_df[(near_otm_puts_df['zero_bid_cumsum'] <= 2) & (near_otm_puts_df['best_bid'] > 0)]

near_otm_puts_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
6750,0.0105,0.0125,0,0
6500,0.0065,0.0070,0,0
6250,0.0035,0.0050,0,0
6000,0.0025,0.0030,0,0
5500,0.0015,0.0020,0,0
5000,0.0005,0.0015,0,0
4500,0.0005,0.0010,0,0


In [210]:
near_otm_calls_df = near_otm_calls_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
near_otm_calls_df['zero_bid_cumsum'] = near_otm_calls_df['zero_bid'].cumsum()
near_otm_calls_df = near_otm_calls_df[(near_otm_calls_df['zero_bid_cumsum'] <= 2) & (near_otm_calls_df['best_bid'] > 0)]

near_otm_calls_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7250,0.0175,0.0195,0,0
7500,0.0090,0.0105,0,0
7750,0.0050,0.0060,0,0
8000,0.0035,0.0040,0,0
8250,0.0025,0.0035,0,0
8500,0.0015,0.0030,0,0
9000,0.0010,0.0020,0,0


In [211]:
next_otm_puts_df = next_otm_puts_df.sort_index(ascending=False)
next_otm_puts_df = next_otm_puts_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
next_otm_puts_df['zero_bid_cumsum'] = next_otm_puts_df['zero_bid'].cumsum()
next_otm_puts_df = next_otm_puts_df[(next_otm_puts_df['zero_bid_cumsum'] <= 2) & (next_otm_puts_df['best_bid'] > 0)]

next_otm_calls_df = next_otm_calls_df.assign(zero_bid=lambda df: (df['best_bid'] == 0).astype(int))
next_otm_calls_df['zero_bid_cumsum'] = next_otm_calls_df['zero_bid'].cumsum()
next_otm_calls_df = next_otm_calls_df[(next_otm_calls_df['zero_bid_cumsum'] <= 2) & (next_otm_calls_df['best_bid'] > 0)]

In [212]:
next_otm_puts_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
6750,0.0265,0.0305,0,0
6500,0.0180,0.0205,0,0
6250,0.0125,0.0140,0,0
6000,0.0100,0.0120,0,0
5750,0.0060,0.0085,0,0
5500,0.0045,0.0060,0,0
5250,0.0030,0.0045,0,0


In [213]:
next_otm_calls_df

,best_bid,best_ask,zero_bid,zero_bid_cumsum
7250,0.0345,0.0380,0,0
7500,0.0230,0.0275,0,0
7750,0.0155,0.0185,0,0
8000,0.0105,0.0135,0,0
8250,0.0075,0.0105,0,0
8500,0.0055,0.0065,0,0
8750,0.0045,0.0065,0,0
9000,0.0030,0.0050,0,0


### Step 2: Calculate volatility for both near-term and next-term options

Refer to VIX white paper page 8.

In [226]:
near_calc_strikes_df = pd.DataFrame(index=near_prices.index)
near_calc_strikes_df['price'] = (near_otm_puts_df['best_bid'] + near_otm_puts_df['best_ask']) / 2
near_calc_strikes_df['price'] = near_calc_strikes_df.price.combine_first((near_otm_calls_df['best_bid'] + near_otm_calls_df['best_ask']) / 2)
near_calc_strikes_df.at[k0_1] = (near_prices.loc[k0_1].call_price + near_prices.loc[k0_1].put_price) / 2

In [227]:
near_calc_strikes_df = near_calc_strikes_df.dropna()
near_calc_strikes_df

,price
4500,0.00075
5000,0.00100
5500,0.00175
6000,0.00275
6250,0.00425
6500,0.00675
6750,0.01150
7000,0.02850
7250,0.01850
7500,0.00975


In [228]:
next_calc_strikes_df = pd.DataFrame(index=next_prices.index)
next_calc_strikes_df['price'] = (next_otm_puts_df['best_bid'] + next_otm_puts_df['best_ask']) / 2
next_calc_strikes_df['price'] = next_calc_strikes_df.price.combine_first((next_otm_calls_df['best_bid'] + next_otm_calls_df['best_ask']) / 2)
next_calc_strikes_df.at[k0_2] = (next_prices.loc[k0_2].call_price + next_prices.loc[k0_2].put_price) / 2

In [229]:
next_calc_strikes_df = next_calc_strikes_df.dropna()
next_calc_strikes_df

,price
5250,0.003750
5500,0.005250
5750,0.007250
6000,0.011000
6250,0.013250
6500,0.019250
6750,0.028500
7000,0.047375
7250,0.036250
7500,0.025250


In [242]:
near_sum = 0
for i in range(len(near_calc_strikes_df)):
    row = near_calc_strikes_df.iloc[i]
    if i == 0:
        deltaKi = near_calc_strikes_df.iloc[i+1].name - row.name
    elif i == len(near_calc_strikes_df) - 1:
        deltaKi = row.name - near_calc_strikes_df.iloc[i-1].name
    else:
        deltaKi = (near_calc_strikes_df.iloc[i+1].name + near_calc_strikes_df.iloc[i-1].name) / 2

    near_sum += deltaKi/(row.name ** 2) * np.e**(R*t1) * row.price
    
next_sum = 0
for i in range(len(next_calc_strikes_df)):
    row = next_calc_strikes_df.iloc[i]
    if i == 0:
        deltaKi = next_calc_strikes_df.iloc[i+1].name - row.name
    elif i == len(next_calc_strikes_df) - 1:
        deltaKi = row.name - next_calc_strikes_df.iloc[i-1].name
    else:
        deltaKi = (next_calc_strikes_df.iloc[i+1].name + next_calc_strikes_df.iloc[i-1].name) / 2
    
    next_sum += deltaKi/(row.name ** 2) * np.e**(R*t2) * row.price
    
sigma1 = ((2/t1) * near_sum) - (1/t1)*((f1/k0_1 - 1)**2)
sigma2 = ((2/t2) * next_sum) - (1/t2)*((f2/k0_2 - 1)**2)

print(sigma1, sigma2)

0.002202498811072921 0.0021638903157043606


In [251]:
XVBT = 100 * np.sqrt(((t1*sigma1**2)*((n2-n)/(n2-n1)) + (t2*sigma2**2)*((n-n1)/(n2-n1)))*(nY/n))

In [252]:
XVBT

0.2181307233835126

## AXVBT and GXVBT Implementation

Refer to *'The Crypto Investor Fear Gauge and the Bitcoin Variance Risk Premium'* by Carol Alexander and Arben Imeraj.

In [255]:
omega = ((n2-nY)/(n2-n1))*n

GXVBT = np.sqrt(omega*t1*sigma1 + (1-omega)*t2*sigma2)

In [256]:
GXVBT

4.5673942828749325

In [259]:
sigma1_a = sigma1 * (f1**-2)
sigma2_a = sigma2 * (f2**-2)

AXVBT = np.sqrt(omega*t1*sigma1_a + (1-omega)*t2*sigma2_a)

In [260]:
AXVBT

0.0006524840724514031